# LSTM Bot

## Project Overview

In this project, you will build a chatbot that can converse with you at the command line. The chatbot will use a Sequence to Sequence text generation architecture with an LSTM as it's memory unit. You will also learn to use pretrained word embeddings to improve the performance of the model. At the conclusion of the project, you will be able to show your chatbot to potential employers.

Additionally, you have the option to use pretrained word embeddings in your model. We have loaded Brown Embeddings from Gensim in the starter code below. You can compare the performance of your model with pre-trained embeddings against a model without the embeddings.



---



A sequence to sequence model (Seq2Seq) has two components:
- An Encoder consisting of an embedding layer and LSTM unit.
- A Decoder consisting of an embedding layer, LSTM unit, and linear output unit.

The Seq2Seq model works by accepting an input into the Encoder, passing the hidden state from the Encoder to the Decoder, which the Decoder uses to output a series of token predictions.

## Dependencies

- Pytorch
- Numpy
- Pandas
- NLTK
- Gzip
- Gensim


Please choose a dataset from the Torchtext website. We recommend looking at the Squad dataset first. Here is a link to the website where you can view your options:

- https://pytorch.org/text/stable/datasets.html





In [1]:
!pip install torchdata==0.3.0

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 47 kB 3.0 MB/s eta 0:00:011


In [1]:
import gensim
import nltk
import numpy as np
import pandas as pd
import torch
import random
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from nltk.corpus import brown
from torchtext.datasets import SQuAD1
from collections import defaultdict
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from torch import nn
from torch.utils.tensorboard import SummaryWriter

In [2]:
def load_df(train_iter):
    '''
    This function loads the dataset into a Pandas DataFrame for processing.
    '''
    questions = []
    answers = []

    for index, (context, question, ans, indices) in enumerate(train_iter):
        if ans[0]:
            questions.append(question)
            answers.append(ans[0])

    df = pd.DataFrame({"question": questions, "answer": answers})
    return df


In [3]:
random.seed(42)
train_data = SQuAD1(root='.', split=('train'))
train_data = load_df(train_data)

In [4]:
class Vocab:
    def __init__(self, name, trimMinValue, trimMaxValue):
        self.name = name
        self.index = {0:"<sos>", 1:"<eos>", 2:"<pad>", 3:"<unk>"}
        self.words = {"<sos>":0, "<eos>":1, "<pad>":2, "<unk>":3}
        self.wordsCounter = {"<sos>":0, "<eos>":0, "<pad>":0, "<unk>":0}
        self.count = 4
        self.tokenizer = RegexpTokenizer(r'\w+')
        self.stemmer = PorterStemmer()
        self.trimMinValue = trimMinValue
        self.trimMaxValue = trimMaxValue
    
    def indexWord(self, word):
        if word not in self.words:
            self.words[word] = self.count
            self.wordsCounter[word] = 1
            self.index[self.count] = word
            self.count += 1
        else:
            self.wordsCounter[word] += 1
    
    def addSentence(self, sentence, maxlen):
        tokens = self.tokenizeSentence(sentence)
        for token in tokens[: maxlen-2]:
            self.indexWord(token)
    
    def tokenizeSentence(self, sentence): 
        ss = sentence.lower()
        ss = self.tokenizer.tokenize(ss)
        ss = [self.stemmer.stem(w) for w in ss if not w.isdigit()]
        return ss
    
    def trimVocab(self):
        trimmedIndex = {0:"<sos>", 1:"<eos>", 2:"<pad>", 3:"<unk>"}
        trimmedWords = {"<sos>":0, "<eos>":1, "<pad>":2, "<unk>":3}
        trimmedWordsCounter = {"<sos>":0, "<eos>":0, "<pad>":0, "<unk>":0}
        trimmedCount = 4
        for i in range(4,self.count):
            if (self.wordsCounter[self.index[i]] >= self.trimMinValue) and (self.wordsCounter[self.index[i]] <= self.trimMaxValue):
                trimmedWords[self.index[i]] = trimmedCount
                trimmedWordsCounter[self.index[i]] = self.wordsCounter[self.index[i]]
                trimmedIndex[trimmedCount] = self.index[i]
                trimmedCount += 1
        self.index = trimmedIndex
        self.words = trimmedWords
        self.wordsCounter = trimmedWordsCounter
        self.count = trimmedCount
    
        
    def prepareSentence(self, sentence):
        tokens = self.tokenizeSentence(sentence)
        sentence = []
        for token in tokens:
            if token in self.words:
                sentence.append(token)
            else:
                sentence.append("<unk>")
        sentence.insert(0, "<sos>")
        sentence.append("<eos>")
        
        return sentence
    
    def paddSentence(self, sentence, maxlen):
        paddedSentence = []
        if len(sentence)>maxlen:
            for i in range(maxlen-1):
                paddedSentence.append(sentence[i])
            paddedSentence.append("<eos>")
        elif len(sentence)<=maxlen:
            paddedSentence = sentence
            for i in range(len(sentence),maxlen):
                paddedSentence.append("<pad>")
        return paddedSentence
    
    def paddSentences(self, sentences, maxlen):
        paddedSentences = []
        for sentence in sentences:
            paddedSentences.append(self.paddSentence(sentence, maxlen))
        return paddedSentences
        
    def indexSentence(self,sentence):
        return [self.words[w] for w in sentence]
    
    def wordSentence(self,sentence):
        return [self.index[w] for w in sentence]


In [5]:
vocab = Vocab(name='SQuAD1Vocab', trimMinValue = 10, trimMaxValue = 1500)
maxlenQ = 10
maxlenA = 5


In [6]:
for i, r in train_data.iterrows():
    question_text = vocab.addSentence(r["question"], maxlenQ)
    answer_text = vocab.addSentence(r["answer"], maxlenA)
print("Added {} words to our vocabulary".format(vocab.count))


Added 32990 words to our vocabulary


In [7]:
vocab.trimVocab()
print("Remain {} words in our vocabulary after trimming".format(vocab.count))


Remain 5458 words in our vocabulary after trimming


In [8]:
trainQ = []
trainA = []

for i,r, in train_data.iterrows():
    trainQ.append(vocab.prepareSentence(r["question"]))
    trainA.append(vocab.prepareSentence(r["answer"]))


In [9]:
print(trainQ[3])
print(trainA[3])
#Padd sentences
trainQ = vocab.paddSentences(trainQ, maxlenQ)
trainA = vocab.paddSentences(trainA, maxlenA)
print(trainQ[3])
print(trainA[3])


['<sos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', 'notr', 'dame', '<eos>']
['<sos>', '<unk>', 'marian', 'place', '<unk>', 'prayer', '<unk>', 'reflect', '<eos>']
['<sos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', 'notr', 'dame', '<eos>', '<pad>']
['<sos>', '<unk>', 'marian', 'place', '<eos>']


In [10]:
trainQtok = []
trainAtok = []

for sentence in trainQ:
    trainQtok.append(vocab.indexSentence(sentence))
for sentence in trainA:
    trainAtok.append(vocab.indexSentence(sentence))

print(trainQtok[3])


[0, 3, 3, 3, 3, 3, 10, 11, 1, 2]


In [11]:
def get_batches(questions, answers, batch_size):
    n_batches = len(questions) // batch_size

    # Consider only full batches
    questions = questions[:n_batches * batch_size]
    answers = answers[:n_batches * batch_size]

    for idx in range(0, len(questions), batch_size):
        questions_batch = np.array(questions[idx:idx + batch_size])
        answers_batch = np.array(answers[idx:idx + batch_size])
        yield questions_batch, answers_batch


In [13]:
class Encoder(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, num_layers, p):
        super(Encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.num_layers = num_layers

        self.dropout = nn.Dropout(p)
        self.embedding = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers, dropout=p, batch_first=True)

    def forward(self, i):
        embedding = self.dropout(self.embedding(i))
        output, (hidden, cell) = self.lstm(embedding)
        return hidden, cell


class Decoder(nn.Module):
      
    def __init__(self, hidden_size, embedding_size, output_size, num_layers, p):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.dropout = nn.Dropout(p)
        self.embedding = nn.Embedding(self.output_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers, dropout=p, batch_first=True)
        self.fc = nn.Linear(self.hidden_size, self.output_size)    
        
    def forward(self, i, hidden, cell):
        i = i.unsqueeze(-1)
        embedding = self.dropout(self.embedding(i))
        output, (hidden, cell) = self.lstm(embedding, (hidden, cell))
        prediction = self.fc(output.squeeze(1))
        return prediction, hidden, cell
        

class LSTMSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder, output_size, device):
        super(LSTMSeq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.output_size = output_size
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio):
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.output_size

        prediction = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)

        input = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            prediction[:, t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[:, t] if teacher_force else top1

        return prediction


In [14]:
num_epochs = 200
batch_size = 256
learning_rate = 0.0001

input_size = vocab.count
output_size = vocab.count
embedding_size = 256
hidden_size = 1024
num_layers = 2
p_dropout = 0.5
teacher_forcing_ratio = 0.5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

enc = Encoder(input_size, hidden_size, embedding_size, num_layers, p_dropout)
dec = Decoder(hidden_size, embedding_size, output_size, num_layers, p_dropout)

model = LSTMSeq2Seq(enc, dec, output_size, device).to(device)
print(model)

cuda
LSTMSeq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(5458, 256)
    (lstm): LSTM(256, 1024, num_layers=2, batch_first=True, dropout=0.5)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(5458, 256)
    (lstm): LSTM(256, 1024, num_layers=2, batch_first=True, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=5458, bias=True)
  )
)


In [15]:
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index = vocab.words['<pad>'])

In [16]:
lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=5)

In [17]:
def train(model, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    l = 0
    
    for x, y in get_batches(trainQtok, trainAtok, batch_size):
        inputs = torch.from_numpy(x).to(device)
        targets = torch.from_numpy(y).to(device)

        optimizer.zero_grad()
        
        output = model(inputs, targets, teacher_forcing_ratio)
        
        loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), targets[:, 1:].reshape(-1))
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        l += 1

        if l == 1:
            for i in range(5):
                target = targets[i].detach().cpu().numpy()
                answer = output[i].argmax(dim=1).detach().cpu().numpy()
                print("Answer: " + str(vocab.wordSentence(answer)) + " | " + "Target: " + str(vocab.wordSentence(target)))
        
    return epoch_loss / l


In [18]:
#Tensorboard
writer = SummaryWriter(log_dir="./runs")

In [19]:
best_train_loss = float('inf')
clip = 1
save_path = 'chatbot_model.pt'

for epoch in range(num_epochs):
    train_loss = train(model, optimizer, criterion, clip)
    
    if train_loss < best_train_loss:
        best_train_loss = train_loss
        torch.save(model.state_dict(), save_path)

    print(f"Epoch: {epoch} | TrainLoss: {train_loss} | BestTrainLoss: {best_train_loss}")
    writer.add_scalar("Loss/train", train_loss, epoch)
    
    lr_scheduler.step(train_loss)
    
writer.flush()
writer.close()


Answer: ['<sos>', 'financ', 'spot', 'n', 'displac'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', 'affirm', 'rough', 'napoleon', 'napoleon'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', 'sherman', 'sherman', 'vocabulari', 'parallel'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', 'jazz', 'california', 'english', 'rousseau'] | Target: ['<sos>', '<unk>', 'marian', 'place', '<eos>']
Answer: ['<sos>', 'rca', 'napoleon', 'iraq', 'kazakhstan'] | Target: ['<sos>', '<unk>', 'golden', 'statu', '<eos>']
Epoch: 0 | TrainLoss: 4.4051996734407215 | BestTrainLoss: 4.4051996734407215
Answer: ['<sos>', '<unk>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<eos>', '<unk>', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['

Epoch: 13 | TrainLoss: 3.9306540984159324 | BestTrainLoss: 3.9306540984159324
Answer: ['<sos>', '<unk>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<unk>', '<eos>', '<unk>', '<eos>'] | Target: ['<sos>', '<unk>', 'marian', 'place', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', '<unk>', 'golden', 'statu', '<eos>']
Epoch: 14 | TrainLoss: 3.907412226437128 | BestTrainLoss: 3.907412226437128
Answer: ['<sos>', '<unk>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<eos>', '<unk>', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>',

Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<eos>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'marian', 'place', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'golden', 'statu', '<eos>']
Epoch: 27 | TrainLoss: 3.6253734143853884 | BestTrainLoss: 3.6253734143853884
Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<eos>', '<eos>', '<eo

Epoch: 40 | TrainLoss: 3.3641305785430107 | BestTrainLoss: 3.3641305785430107
Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<eos>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'marian', 'place', '<eos>']
Answer: ['<sos>', '<unk>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'golden', 'statu', '<eos>']
Epoch: 41 | TrainLoss: 3.3380147344187687 | BestTrainLoss: 3.3380147344187687
Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>

Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<eos>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', '<unk>', 'marian', 'place', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'golden', 'statu', '<eos>']
Epoch: 54 | TrainLoss: 3.0674579415405008 | BestTrainLoss: 3.0674579415405008
Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<eo

...

Answer: ['<sos>', '<unk>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', 'statu', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', 'medal', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<eos>', '<eos>', 'place', '<eos>'] | Target: ['<sos>', '<unk>', 'marian', 'place', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'golden', 'statu', '<eos>']
Epoch: 147 | TrainLoss: 1.3139428301164282 | BestTrainLoss: 1.3139428301164282
Answer: ['<sos>', '<unk>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', 'statu', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', 'main', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<eos>', 'marian', 'pl

Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', 'statu', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', 'build', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<eos>', 'marian', '<unk>', '<eos>'] | Target: ['<sos>', '<unk>', 'marian', 'place', '<eos>']
Answer: ['<sos>', '<unk>', '<eos>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'golden', 'statu', '<eos>']
Epoch: 160 | TrainLoss: 1.138488338181847 | BestTrainLoss: 1.138488338181847
Answer: ['<sos>', '<unk>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', 'build', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<eos>', 'marian', 'pl

Answer: ['<sos>', '<unk>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', 'statu', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', 'main', 'build', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<eos>', '<unk>', 'ice', '<eos>'] | Target: ['<sos>', '<unk>', 'marian', 'place', '<eos>']
Answer: ['<sos>', '<unk>', 'golden', 'statu', '<eos>'] | Target: ['<sos>', '<unk>', 'golden', 'statu', '<eos>']
Epoch: 173 | TrainLoss: 0.9753409707755373 | BestTrainLoss: 0.9753409707755373
Answer: ['<sos>', '<unk>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', 'statu', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<unk>', 'marian', 'pla

Answer: ['<sos>', '<unk>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', 'statu', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', 'build', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<unk>', 'marian', '<eos>', '<eos>'] | Target: ['<sos>', '<unk>', 'marian', 'place', '<eos>']
Answer: ['<sos>', '<unk>', 'standard', 'statu', '<eos>'] | Target: ['<sos>', '<unk>', 'golden', 'statu', '<eos>']
Epoch: 186 | TrainLoss: 0.8083804643642135 | BestTrainLoss: 0.8083804643642135
Answer: ['<sos>', '<unk>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', 'statu', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', 'age', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<eos>', 'marian', 

Answer: ['<sos>', '<unk>', '<unk>', '<unk>', '<eos>'] | Target: ['<sos>', 'saint', '<unk>', '<unk>', '<eos>']
Answer: ['<sos>', '<unk>', '<unk>', 'statu', '<eos>'] | Target: ['<sos>', '<unk>', 'copper', 'statu', '<eos>']
Answer: ['<sos>', '<unk>', 'main', 'build', '<eos>'] | Target: ['<sos>', '<unk>', 'main', 'build', '<eos>']
Answer: ['<sos>', '<eos>', 'marian', 'place', '<eos>'] | Target: ['<sos>', '<unk>', 'marian', 'place', '<eos>']
Answer: ['<sos>', '<unk>', 'golden', 'statu', '<eos>'] | Target: ['<sos>', '<unk>', 'golden', 'statu', '<eos>']
Epoch: 199 | TrainLoss: 0.6897831955674099 | BestTrainLoss: 0.6897831955674099
